In [281]:
from datetime import datetime 
import pandas as pd
import numpy as np
import re
import emoji

# The format string should match the format of the date and time string
# In this case, the date and time string is in the format 'MM/DD/YY, HH:MM:SS AM/PM'
date_time_format = '%m/%d/%y, %I:%M:%S %p'
date_time_list = []
name_list = []
text_list = []
total_file = []

file = r"C:\Users\markkw\Downloads\WhatsApp Chat\_chat.txt"
with open(file, encoding="utf8") as f:
    for line in f.readlines():
        total_file.append(line)
        if line[0] == "[":
            date_time_list.append(datetime.strptime(line.split(']', 1)[0].replace('[', ''), date_time_format))
            name_list.append(line.split(']', 1)[1].split(":",1)[0])
            if ":" in line.split(']', 1)[1]:
                text_list.append(line.split(']', 1)[1].split(":",1)[1].replace("\n", ""))
            else: 
                text_list.append("-")

df = pd.DataFrame()
df['date'] = date_time_list
df['Person'] = name_list
df['Message'] = text_list
df = df[df['Message'] != "-"]
pattern = re.compile(r"|".join(map(re.escape, emoji.EMOJI_DATA)))

df["Emojis"] = df["Message"].apply(lambda x: "".join(pattern.findall(x)))
df["Sentence"] = df["Message"].apply(lambda x: pattern.sub("", x))

# start_date = '2022-01-01 00:01:53'
# end_date = '2022-12-10 16:08:33'
# Select DataFrame rows between two dates
df = df[(df['date'] > '2022-1-1') & (df['date'] <= '2022-12-30')]



In [320]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
import re

def sentiment_analyzer(row):
    score = analyser.polarity_scores(str(row))
    
    return score['compound']

def count_emojis(row):
    return len(row)

def count_char(row):
    return len(row.replace(" ", ""))

def count_words(row):
    return len(row.split(" ")) - 1 

def count_ha(row):
    if row.count('haha') == 1:
        return 2
    elif row.count('hahaha') == 1:
        return 3
    elif row.count('hahah') == 1:
        return 2.5
    # elif emoji.EMOJI_DATA[row[0]]['en'] == ':face_with_tears_of_joy:':
    # # elif '\U0001f602' in row:
    # #     # emoticons = re.finditer(ru'[\U0001f602]', row)
    # #     # count = sum(1 for _ in emoticons)
    #     return 3
    else:
        return row.count('haha') * 2

def count_quest(row):
    return row.count('?')

def count_upper(row):
    return sum(1 for char in row if char.isupper())


In [326]:
emoji.EMOJI_DATA['😂']['en']

':face_with_tears_of_joy:'

In [321]:
df['Person'] = df['Person'].replace(' \u202a+31\xa06\xa055442275\u202c', " Chris 'netjes' de Kwaasteniet")
df['Person'] = df['Person'].replace(" Chris 'netjes' de Kwaasteniet", "Chris 'netjes' de Kwaasteniet")
df['Person'] = df['Person'].replace(' \u200eYou', " Mark de Kw")
df['Person'] = df['Person'].replace(' Mark de Kw', "Mark de Kw")
df['Emojis_count'] = df['Emojis'].map(lambda x: count_emojis(x))
df['character_count'] = df['Sentence'].map(lambda x: count_char(x))
df['word_count'] = df['Sentence'].map(lambda x: count_words(x))
df['haha_count'] = df['Sentence'].map(lambda x: count_ha(x))
df['quest_count'] = df['Sentence'].map(lambda x: count_quest(x))
df['upper_count'] = df['Sentence'].map(lambda x: count_upper(x))
df['haha_count'] = df['haha_count'].replace(0, np.nan)
df['Sentiment'] = df_test_en['Sentence'].iloc[-7520:].values
df['Sentiment_score'] = df['Sentiment'].map(lambda x: sentiment_analyzer(x))


# df['Sentiment'] = df.apply(lambda row : sentiment_analyzer(row), axis=1)
# df['Emojis'].value_counts()

In [292]:
df

,date,Person,Message,Emojis,Sentence,Emojis_count,character_count,word_count,haha_count,quest_count,upper_count,Sentiment,Sentiment_score
24956,2022-01-01 00:01:53,Aleks 'after' Gatti,De ebsre wensen allemal,,De ebsre wensen allemal,0,20,4,NaN,0,1,The ebsre wishes all,0.1531
24957,2022-01-01 00:01:57,Thushara,Hahahahahaga,,Hahahahahaga,0,12,1,3.0,0,1,hahahahahaga,0.0000
24958,2022-01-01 00:02:11,Thushara,Omg auw,,Omg auw,0,6,2,NaN,0,1,Omg ouch,0.0000
24959,2022-01-01 00:02:35,Thushara,"Gelukkig nieuwjaar, op een jaar van weer alle...",❤️,"Gelukkig nieuwjaar, op een jaar van weer alle...",2,60,14,NaN,0,3,"Happy New Year, to a year of ignoring everyth...",0.2500
24960,2022-01-01 00:10:06,Omar 'ringopener' van der Heijden,Happy new year boyos and dame! 😘❤️,😘❤️,Happy new year boyos and dame!,3,25,7,NaN,0,1,Happy new year boyos and lady!,0.6114
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32473,2022-12-10 15:48:59,Steven 'joppie' Oudejans,Amsterdamse invalkracht dwong kinderen horror...,,Amsterdamse invalkracht dwong kinderen horror...,0,72,10,NaN,0,3,Amsterdam substitute force forced children to...,-0.7717
32474,2022-12-10 15:49:17,Steven 'joppie' Oudejans,Vibes,,Vibes,0,5,1,NaN,0,1,vibes,0.0000
32475,2022-12-10 15:56:24,Mark de Kw,Werknemers te kort pik,,Werknemers te kort pik,0,19,4,NaN,0,1,Employees too short cock,-0.1531
32476,2022-12-10 16:08:28,Nino 'caretaker' Gatti,Hahahahahaha,,Hahahahahaha,0,12,1,3.0,0,1,Hahahahahaha,0.0000


In [293]:
df.iloc[1:,:].Person.value_counts()

 Peer 'altijd high' Rietveld          1055
 Thushara                             1012
 Steven 'joppie' Oudejans              901
 Nino 'caretaker' Gatti                753
 Bob '3e dimensie' Oudejans            751
 Lukas 'Stimorol' Baars                667
 Omar 'ringopener' van der Heijden     506
 David 'hele J' Besson                 465
 Aleks 'after' Gatti                   375
Mark de Kw                             372
Chris 'netjes' de Kwaasteniet          336
 Pieter Spuijbroek                     171
 Luuk De Kwaasteniet                   155
Name: Person, dtype: int64

In [294]:
df.iloc[1:,:].groupby(by="Person").sum().quest_count

Person
 Aleks 'after' Gatti                   34
 Bob '3e dimensie' Oudejans           113
 David 'hele J' Besson                 98
 Lukas 'Stimorol' Baars               125
 Luuk De Kwaasteniet                   16
 Nino 'caretaker' Gatti                85
 Omar 'ringopener' van der Heijden     85
 Peer 'altijd high' Rietveld          112
 Pieter Spuijbroek                     12
 Steven 'joppie' Oudejans             129
 Thushara                              94
Chris 'netjes' de Kwaasteniet          71
Mark de Kw                             41
Name: quest_count, dtype: int64

In [295]:
df.iloc[1:,:].groupby(by="Person").count()

,date,Message,Emojis,Sentence,Emojis_count,character_count,word_count,haha_count,quest_count,upper_count,Sentiment,Sentiment_score
Person,,,,,,,,,,,,
Aleks 'after' Gatti,375,375,375,375,375,375,375,31,375,375,363,375
Bob '3e dimensie' Oudejans,751,751,751,751,751,751,751,56,751,751,737,751
David 'hele J' Besson,465,465,465,465,465,465,465,9,465,465,456,465
Lukas 'Stimorol' Baars,667,667,667,667,667,667,667,9,667,667,647,667
Luuk De Kwaasteniet,155,155,155,155,155,155,155,11,155,155,153,155
Nino 'caretaker' Gatti,753,753,753,753,753,753,753,71,753,753,729,753
Omar 'ringopener' van der Heijden,506,506,506,506,506,506,506,41,506,506,491,506
Peer 'altijd high' Rietveld,1055,1055,1055,1055,1055,1055,1055,44,1055,1055,1024,1055
Pieter Spuijbroek,171,171,171,171,171,171,171,8,171,171,160,171


In [296]:
df.iloc[1:,:].groupby(by="Person").mean().upper_count

Person
 Aleks 'after' Gatti                  1.106667
 Bob '3e dimensie' Oudejans           2.047936
 David 'hele J' Besson                1.593548
 Lukas 'Stimorol' Baars               1.616192
 Luuk De Kwaasteniet                  1.303226
 Nino 'caretaker' Gatti               1.467463
 Omar 'ringopener' van der Heijden    1.478261
 Peer 'altijd high' Rietveld          1.363033
 Pieter Spuijbroek                    1.812865
 Steven 'joppie' Oudejans             1.572697
 Thushara                             1.658103
Chris 'netjes' de Kwaasteniet         1.553571
Mark de Kw                            1.440860
Name: upper_count, dtype: float64

In [297]:
df.iloc[1:,:].groupby(by="Person").sum().upper_count

Person
 Aleks 'after' Gatti                   415
 Bob '3e dimensie' Oudejans           1538
 David 'hele J' Besson                 741
 Lukas 'Stimorol' Baars               1078
 Luuk De Kwaasteniet                   202
 Nino 'caretaker' Gatti               1105
 Omar 'ringopener' van der Heijden     748
 Peer 'altijd high' Rietveld          1438
 Pieter Spuijbroek                     310
 Steven 'joppie' Oudejans             1417
 Thushara                             1678
Chris 'netjes' de Kwaasteniet          522
Mark de Kw                             536
Name: upper_count, dtype: int64

In [298]:
df.iloc[1:,:].groupby(by="Person").mean().Sentiment_score

Person
 Aleks 'after' Gatti                  0.175175
 Bob '3e dimensie' Oudejans           0.172193
 David 'hele J' Besson                0.186392
 Lukas 'Stimorol' Baars               0.122257
 Luuk De Kwaasteniet                  0.159242
 Nino 'caretaker' Gatti               0.135768
 Omar 'ringopener' van der Heijden    0.246587
 Peer 'altijd high' Rietveld          0.108241
 Pieter Spuijbroek                    0.207200
 Steven 'joppie' Oudejans             0.130074
 Thushara                             0.104741
Chris 'netjes' de Kwaasteniet         0.170219
Mark de Kw                            0.152819
Name: Sentiment_score, dtype: float64

In [299]:
df.iloc[1:,:].groupby(by="Person").sum().Sentiment_score

Person
 Aleks 'after' Gatti                   65.6907
 Bob '3e dimensie' Oudejans           129.3169
 David 'hele J' Besson                 86.6722
 Lukas 'Stimorol' Baars                81.5453
 Luuk De Kwaasteniet                   24.6825
 Nino 'caretaker' Gatti               102.2333
 Omar 'ringopener' van der Heijden    124.7729
 Peer 'altijd high' Rietveld          114.1938
 Pieter Spuijbroek                     35.4312
 Steven 'joppie' Oudejans             117.1969
 Thushara                             105.9974
Chris 'netjes' de Kwaasteniet          57.1935
Mark de Kw                             56.8486
Name: Sentiment_score, dtype: float64

In [300]:
df.iloc[1:,:].groupby(by="Person").sum().word_count

Person
 Aleks 'after' Gatti                  2569
 Bob '3e dimensie' Oudejans           8741
 David 'hele J' Besson                4114
 Lukas 'Stimorol' Baars               6688
 Luuk De Kwaasteniet                  1274
 Nino 'caretaker' Gatti               5418
 Omar 'ringopener' van der Heijden    4474
 Peer 'altijd high' Rietveld          7097
 Pieter Spuijbroek                    2089
 Steven 'joppie' Oudejans             8539
 Thushara                             9616
Chris 'netjes' de Kwaasteniet         3926
Mark de Kw                            3028
Name: word_count, dtype: int64

In [303]:
df.iloc[1:,:].groupby(by="Person").mean().word_count

Person
 Aleks 'after' Gatti                   6.850667
 Bob '3e dimensie' Oudejans           11.639148
 David 'hele J' Besson                 8.847312
 Lukas 'Stimorol' Baars               10.026987
 Luuk De Kwaasteniet                   8.219355
 Nino 'caretaker' Gatti                7.195219
 Omar 'ringopener' van der Heijden     8.841897
 Peer 'altijd high' Rietveld           6.727014
 Pieter Spuijbroek                    12.216374
 Steven 'joppie' Oudejans              9.477248
 Thushara                              9.501976
Chris 'netjes' de Kwaasteniet         11.684524
Mark de Kw                             8.139785
Name: word_count, dtype: float64

In [304]:
df.iloc[1:,:].groupby(by="Person").mean().haha_count

Person
 Aleks 'after' Gatti                  2.161290
 Bob '3e dimensie' Oudejans           2.035714
 David 'hele J' Besson                2.111111
 Lukas 'Stimorol' Baars               2.000000
 Luuk De Kwaasteniet                  2.045455
 Nino 'caretaker' Gatti               2.084507
 Omar 'ringopener' van der Heijden    2.048780
 Peer 'altijd high' Rietveld          2.272727
 Pieter Spuijbroek                    2.250000
 Steven 'joppie' Oudejans             2.111111
 Thushara                             2.897959
Chris 'netjes' de Kwaasteniet         2.000000
Mark de Kw                            2.000000
Name: haha_count, dtype: float64

In [322]:
df.iloc[1:,:].groupby(by="Person").sum().haha_count

Person
 Aleks 'after' Gatti                   67.0
 Bob '3e dimensie' Oudejans           114.0
 David 'hele J' Besson                 19.0
 Lukas 'Stimorol' Baars                18.0
 Luuk De Kwaasteniet                   22.5
 Nino 'caretaker' Gatti               148.0
 Omar 'ringopener' van der Heijden     84.0
 Peer 'altijd high' Rietveld          100.0
 Pieter Spuijbroek                     18.0
 Steven 'joppie' Oudejans              76.0
 Thushara                             142.0
Chris 'netjes' de Kwaasteniet          42.0
Mark de Kw                             56.0
Name: haha_count, dtype: float64

In [312]:
df.iloc[1:,:].groupby(by="Person").sum().haha_count

Person
 Aleks 'after' Gatti                   67.0
 Bob '3e dimensie' Oudejans           114.0
 David 'hele J' Besson                 19.0
 Lukas 'Stimorol' Baars                18.0
 Luuk De Kwaasteniet                   22.5
 Nino 'caretaker' Gatti               148.0
 Omar 'ringopener' van der Heijden     84.0
 Peer 'altijd high' Rietveld          100.0
 Pieter Spuijbroek                     18.0
 Steven 'joppie' Oudejans              76.0
 Thushara                             142.0
Chris 'netjes' de Kwaasteniet          42.0
Mark de Kw                             56.0
Name: haha_count, dtype: float64

In [306]:
df.iloc[1:,:].groupby(by="Person").sum().Emojis_count

Person
 Aleks 'after' Gatti                  112
 Bob '3e dimensie' Oudejans           112
 David 'hele J' Besson                116
 Lukas 'Stimorol' Baars               120
 Luuk De Kwaasteniet                   11
 Nino 'caretaker' Gatti               161
 Omar 'ringopener' van der Heijden    211
 Peer 'altijd high' Rietveld          142
 Pieter Spuijbroek                     73
 Steven 'joppie' Oudejans             123
 Thushara                             196
Chris 'netjes' de Kwaasteniet          71
Mark de Kw                             41
Name: Emojis_count, dtype: int64

In [307]:
df.iloc[1:,:].groupby(by="Person").max().word_count

C:\Users\markkw\AppData\Local\Temp\ipykernel_17128\2308823950.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  df.iloc[1:,:].groupby(by="Person").max().word_count


Person
 Aleks 'after' Gatti                  157
 Bob '3e dimensie' Oudejans           124
 David 'hele J' Besson                 72
 Lukas 'Stimorol' Baars               109
 Luuk De Kwaasteniet                  112
 Nino 'caretaker' Gatti                65
 Omar 'ringopener' van der Heijden    121
 Peer 'altijd high' Rietveld           63
 Pieter Spuijbroek                     74
 Steven 'joppie' Oudejans             197
 Thushara                              76
Chris 'netjes' de Kwaasteniet         108
Mark de Kw                            131
Name: word_count, dtype: int64

In [308]:
df.iloc[1:,:].groupby(by="Person").max().character_count

C:\Users\markkw\AppData\Local\Temp\ipykernel_17128\354465570.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  df.iloc[1:,:].groupby(by="Person").max().character_count


Person
 Aleks 'after' Gatti                  699
 Bob '3e dimensie' Oudejans           620
 David 'hele J' Besson                347
 Lukas 'Stimorol' Baars               457
 Luuk De Kwaasteniet                  468
 Nino 'caretaker' Gatti               325
 Omar 'ringopener' van der Heijden    504
 Peer 'altijd high' Rietveld          303
 Pieter Spuijbroek                    374
 Steven 'joppie' Oudejans             926
 Thushara                             328
Chris 'netjes' de Kwaasteniet         459
Mark de Kw                            576
Name: character_count, dtype: int64

In [309]:
lengths = df["Sentence"].str.len()
argmax = np.where(lengths == lengths.max())[0]
print(df.iloc[argmax].values)

[[Timestamp('2022-03-28 11:39:20') " Steven 'joppie' Oudejans"
  " Even een gek vraagje, ken jij wellicht iemand die zich bezighoudt met huurrecht? We kregen gister een beetje een vreemd belletje van onze huisbaas, zij is zelf Italiaans, heeft het huis gekocht toen ze hier woonde en verhuurd het nu aan ons en woont zelf in Doha. We hebben inmiddels een contract voor onbepaalde tijd, maar ze belde gister met een verhaal dat de bank erachter is gekomen dat zij als niet-Nederlandse staatsburger zelf in het buitenland woont en een huis hier verhuurd, dus dat zij 1. het huis moet verkopen of 2. een contract met een maximale duratie van 5 jaar kan aanbieden onder een 'diplomatenclausule'. Ze wilt ons nu dus een nieuw contract laten tekenen waarmee we met terugwerkende kracht nog 4 jaar hier kunnen wonen en zij gedurende de hele duratie van dat contract kan aangeven dat ze terug wilt verhuizen naar Nederland en ons dus ff binnen 3 maanden eruit kan kicken. Voelt niet heel chill en volgens mij

In [183]:
df

,Person,Message,Emojis,Sentence,Sentiment,Sentiment_score,Emojis_count
2015-12-08 21:37:40,"[, 👀🕹Thunder Muffin💰⚡]",‎Messages and calls are end-to-end encrypted....,,‎Messages and calls are end-to-end encrypted....,‎Messages and calls are end-to-end encrypted....,-0.2960,0
2019-08-14 07:58:38,"[Chris, 'netjes' de Kwaasteniet]","Not sure yet, ff vragen :)",,"Not sure yet, ff vragen :)","Not sure yet, just ask :)",0.2589,0
2019-08-14 09:06:03,"[, Bob '3e dimensie' Oudejans]",Hahahaha in Overwatch is er nu een Nederlands...,,Hahahaha in Overwatch is er nu een Nederlands...,Hahahaha in Overwatch there is now a Dutch hero,0.5574,0
2019-08-14 09:06:36,"[, Bob '3e dimensie' Oudejans]",Met o.a. De voice lines:,,Met o.a. De voice lines:,Including The voice lines:,0.0000,0
2019-08-14 09:06:47,"[, Bob '3e dimensie' Oudejans]",En als ie zn ult popt: HET UNIVERSUM ZINGT VO...,,En als ie zn ult popt: HET UNIVERSUM ZINGT VO...,And when he pops his ult: THE UNIVERSE SINGS ...,0.0000,0
...,...,...,...,...,...,...,...
2022-12-10 15:48:59,"[, Steven 'joppie' Oudejans]",Amsterdamse invalkracht dwong kinderen horror...,,Amsterdamse invalkracht dwong kinderen horror...,Amsterdam substitute force forced children to...,-0.7717,0
2022-12-10 15:49:17,"[, Steven 'joppie' Oudejans]",Vibes,,Vibes,vibes,0.0000,0
2022-12-10 15:56:24,"[Mark, de Kw]",Werknemers te kort pik,,Werknemers te kort pik,Employees too short cock,-0.1531,0
2022-12-10 16:08:28,"[, Nino 'caretaker' Gatti]",Hahahahahaha,,Hahahahahaha,Hahahahahaha,0.0000,0


In [146]:
df_check

,Sentiment_score
Person,
Aleks 'after' Gatti,0.164072
Bob '3e dimensie' Oudejans,0.171382
David 'hele J' Besson,0.190581
Lukas 'Stimorol' Baars,0.099040
Luuk De Kwaasteniet,0.155430
Nino 'caretaker' Gatti,0.126986
Omar 'ringopener' van der Heijden,0.227580
Peer 'altijd high' Rietveld,0.106758
Pieter Spuijbroek,0.187852


In [153]:
df_check.index[1]

" Bob '3e dimensie' Oudejans"

In [133]:
df_check.index[2]

" Chris 'netjes' de Kwaasteniet"

In [85]:
df

,Person,Message,Emojis,Sentence,Sentiment,Sentiment_score
2015-12-08 21:37:40,👀🕹Thunder Muffin💰⚡,‎Messages and calls are end-to-end encrypted....,,‎Messages and calls are end-to-end encrypted....,‎Messages and calls are end-to-end encrypted....,-0.2960
2019-08-14 07:58:38,‪+31 6 55442275‬,"Not sure yet, ff vragen :)",,"Not sure yet, ff vragen :)","Not sure yet, just ask :)",0.2589
2019-08-14 09:06:03,Bob '3e dimensie' Oudejans,Hahahaha in Overwatch is er nu een Nederlands...,,Hahahaha in Overwatch is er nu een Nederlands...,Hahahaha in Overwatch there is now a Dutch hero,0.5574
2019-08-14 09:06:36,Bob '3e dimensie' Oudejans,Met o.a. De voice lines:,,Met o.a. De voice lines:,Including The voice lines:,0.0000
2019-08-14 09:06:47,Bob '3e dimensie' Oudejans,En als ie zn ult popt: HET UNIVERSUM ZINGT VO...,,En als ie zn ult popt: HET UNIVERSUM ZINGT VO...,And when he pops his ult: THE UNIVERSE SINGS ...,0.0000
...,...,...,...,...,...,...
2022-12-10 15:48:59,Steven 'joppie' Oudejans,Amsterdamse invalkracht dwong kinderen horror...,,Amsterdamse invalkracht dwong kinderen horror...,Amsterdam substitute force forced children to...,-0.7717
2022-12-10 15:49:17,Steven 'joppie' Oudejans,Vibes,,Vibes,vibes,0.0000
2022-12-10 15:56:24,Mark de Kw,Werknemers te kort pik,,Werknemers te kort pik,Employees too short cock,-0.1531
2022-12-10 16:08:28,Nino 'caretaker' Gatti,Hahahahahaha,,Hahahahahaha,Hahahahahaha,0.0000


In [65]:
df

,Person,Message,Emojis,Sentence,Sentiment
2015-12-08 21:37:40,👀🕹Thunder Muffin💰⚡,‎Messages and calls are end-to-end encrypted....,,‎Messages and calls are end-to-end encrypted....,‎Messages and calls are end-to-end encrypted....
2019-08-14 07:58:38,‪+31 6 55442275‬,"Not sure yet, ff vragen :)",,"Not sure yet, ff vragen :)","Not sure yet, just ask :)"
2019-08-14 09:06:03,Bob '3e dimensie' Oudejans,Hahahaha in Overwatch is er nu een Nederlands...,,Hahahaha in Overwatch is er nu een Nederlands...,Hahahaha in Overwatch there is now a Dutch hero
2019-08-14 09:06:36,Bob '3e dimensie' Oudejans,Met o.a. De voice lines:,,Met o.a. De voice lines:,Including The voice lines:
2019-08-14 09:06:47,Bob '3e dimensie' Oudejans,En als ie zn ult popt: HET UNIVERSUM ZINGT VO...,,En als ie zn ult popt: HET UNIVERSUM ZINGT VO...,And when he pops his ult: THE UNIVERSE SINGS ...
...,...,...,...,...,...
2022-12-10 15:48:59,Steven 'joppie' Oudejans,Amsterdamse invalkracht dwong kinderen horror...,,Amsterdamse invalkracht dwong kinderen horror...,Amsterdam substitute force forced children to...
2022-12-10 15:49:17,Steven 'joppie' Oudejans,Vibes,,Vibes,vibes
2022-12-10 15:56:24,Mark de Kw,Werknemers te kort pik,,Werknemers te kort pik,Employees too short cock
2022-12-10 16:08:28,Nino 'caretaker' Gatti,Hahahahahaha,,Hahahahahaha,Hahahahahaha


In [48]:
df_test_en['Sentence'].values

array([' \u200eMessages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them.',
       ' Not sure yet, just ask :)',
       ' Hahahaha in Overwatch there is now a Dutch hero', ...,
       ' Employees too short cock', ' Hahahahahaha',
       ' I find this humorous'], dtype=object)

In [38]:
df['Sentence'].to_excel("chat_excel.xlsx")

In [39]:
df_test_en = pd.read_excel("chat_excel_en_sent.xlsx")

In [40]:
df_test_en

,Sentence
0,‎Messages and calls are end-to-end encrypted....
1,"Not sure yet, just ask :)"
2,Hahahaha in Overwatch there is now a Dutch hero
3,Including The voice lines:
4,And when he pops his ult: THE UNIVERSE SINGS ...
...,...
32466,Amsterdam substitute force forced children to...
32467,vibes
32468,Employees too short cock
32469,Hahahahahaha
